# Dataloading into Graph Neural Network / EdgeClassifer stage

**Goal**: Test the weighting and hard cut config of the data loading process

In [1]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd
import yaml

from time import time as tt



### Roadmap

1. Inspect a set of raw Athena files (i.e. hits, clusters, particles)
2. Run the functional implementation on these files and inspect the results
3. Build a class that performs the same functionality
4. Test the class on the same files
5. Find ways to refactor and simplify for maximum user-friendliness!

In [ ]:
from gnn4itk_cf.stages.edge_classifier import InteractionGNN